In [1]:
import os


In [2]:
%pwd

'f:\\Personal Project\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'f:\\Personal Project\\Kidney-Disease-Classification-Deep-Learning-Project'

Update Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path


#From this can access these are the variables from other file.Consider as an entity. Frozen means cannot change variable values
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path



In [11]:
from src.cnn_classifier.constants import *
from src.cnn_classifier.utils.common import read_yaml,create_directories

# print(CONFIG_FILE_PATH)

Confuguration Manager

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [22]:
import os
import zipfile
import gdown
from src.cnn_classifier import logger
from src.cnn_classifier.utils.common import get_size

Update components

In [23]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self)->str:
        """
        fetch data from url
        """
        try:
            dataset_url=self.config.source_url
            zip_download_dir=self.config.local_data_file
            os.makedirs("artifacts/data-ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id=dataset_url.split('/')[-2]
            prefix='https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        Zip_fiel_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    

Pipeline

In [25]:
try:
    config=ConfigurationManager()
    data_ingesion_config=config.get_data_ingestion_config()
    data_ingesion=DataIngestion(config=data_ingesion_config)
    data_ingesion.download_file()
    data_ingesion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-25 15:36:03,968: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-25 15:36:03,970: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-25 15:36:03,971: INFO: common: created directory at: artifacts]
[2024-02-25 15:36:04,092: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-25 15:36:04,094: INFO: 992390727: Downloading data from https://drive.google.com/file/d/1KldrdOXZUjRkEgWwG8D4vDHhYJZV0Sde/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1KldrdOXZUjRkEgWwG8D4vDHhYJZV0Sde
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1KldrdOXZUjRkEgWwG8D4vDHhYJZV0Sde&confirm=t&uuid=ecda7f5b-3077-4d6a-8de1-ce65433daa5a
To: f:\Personal Project\Kidney-Disease-Classification-Deep-Learning-Project\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:22<00:00, 2.56MB/s]

[2024-02-25 15:36:29,615: INFO: 992390727: Downloaded data from https://drive.google.com/file/d/1KldrdOXZUjRkEgWwG8D4vDHhYJZV0Sde/view?usp=sharing into file artifacts/data_ingestion/data.zip]
